In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Just_normal/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'PV_data.csv'
PV_data = pd.read_csv(path + filename)

In [ ]:
set_index(PV_data)

In [ ]:
data = pd.DataFrame(columns=['garch', 'return'], index=PV_data.index)

In [ ]:
data['garch'] = PV_data["('GARCH', 'normal', 0)"].values

In [ ]:
data['return'] = PV_data["log_returns"].values

In [ ]:
data['PV'] = PV_data["PV"].values

In [ ]:
data

,garch,return,PV
Date,,,
2000-02-02,0.800313,-0.027564,1.048503
2000-02-03,0.655577,0.893688,0.882277
2000-02-04,0.491617,-0.052641,0.808791
2000-02-07,0.408321,-0.025746,0.794178
2000-02-08,0.339040,0.976294,0.698442
...,...,...,...
2024-08-26,0.093281,-0.275848,0.102624
2024-08-27,-0.162765,0.111656,0.099215
2024-08-28,-0.126100,-0.507014,0.075804


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## PV

In [ ]:
daily_train, daily_val, daily_test = split_data(data, 'PV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
PV_X_train, PV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
PV_X_val, PV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
PV_X_test, PV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# PV_X_train = PV_X_train.reshape(PV_X_train.shape[0], win_sz, -1)
# PV_X_val = PV_X_val.reshape(PV_X_val.shape[0], win_sz, -1)
# PV_X_test = PV_X_test.reshape(PV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(PV_X_train.shape[1], PV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,037 (54.83 KB)

 Trainable params: 14,037 (54.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(PV_X_train, PV_y_train, epochs=80, batch_size=32, validation_data=(PV_X_val, PV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - loss: 0.3049 - val_loss: 0.2457
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 0.0788 - val_loss: 0.1045
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0726 - val_loss: 0.0909
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 0.0659 - val_loss: 0.1749
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0617 - val_loss: 0.1468
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 0.0578 - val_loss: 0.2108
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.0468 - val_loss: 0.1756
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0473 - val_loss: 0.1755
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - loss: 0.0447 - val_loss: 0.1585
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0551 - val_loss: 0.2320
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0452 - val_loss: 0.1320
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Predictions
PV_test_predictions = lstm_garch_model.predict(PV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step


In [ ]:
mse = mean_squared_error(PV_y_test, PV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(PV_y_test, PV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(PV_y_test, PV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.055988230171126516
RMSE: 0.2366183217147956
MAE: 0.18159582683793576
MAPE: 85.42000358472556%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(PV_X_train.shape[1], PV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,741 (41.96 KB)

 Trainable params: 10,741 (41.96 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(PV_X_train, PV_y_train, epochs=80, batch_size=32, validation_data=(PV_X_val, PV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 38ms/step - loss: 0.2165 - val_loss: 0.1910
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 0.0958 - val_loss: 0.1258
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0766 - val_loss: 0.1271
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 0.0584 - val_loss: 0.1062
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - loss: 0.0819 - val_loss: 0.1702
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 0.0521 - val_loss: 0.1023
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.0572 - val_loss: 0.1360
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 0.0443 - val_loss: 0.2256
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0477 - val_loss: 0.2129
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0420 - val_loss: 0.1460
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 0.0385 - val_loss: 0.1559
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━

In [34]:
# Predictions
PV_gru_predictions = gru_model.predict(PV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [35]:
mse = mean_squared_error(PV_y_test, PV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(PV_y_test, PV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(PV_y_test, PV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.20733563436604965
RMSE: 0.4553412284935877
MAE: 0.31673263290788733
MAPE: 149.45156369175734%


### LSTM_GRU

In [36]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(PV_X_train.shape[1], PV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
lstm_gru_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,045 (50.96 KB)

 Trainable params: 13,045 (50.96 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
lstm_gru_history = lstm_garch_model.fit(PV_X_train, PV_y_train, epochs=80, batch_size=32, validation_data=(PV_X_val, PV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0191 - val_loss: 0.1544
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - loss: 0.0143 - val_loss: 0.2413
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0134 - val_loss: 0.1731
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - loss: 0.0179 - val_loss: 0.1937
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0195 - val_loss: 0.1682
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - loss: 0.0169 - val_loss: 0.1694
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 0.0170 - val_loss: 0.1533
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0169 - val_loss: 0.1482
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0140 - val_loss: 0.1953
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 0.0189 - val_loss: 0.2138
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0221 - val_loss: 0.1596
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [39]:
# Predictions
PV_lstm_gru_predictions = lstm_garch_model.predict(PV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [40]:
mse = mean_squared_error(PV_y_test, PV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(PV_y_test, PV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(PV_y_test, PV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.050441956919231326
RMSE: 0.2245928692528579
MAE: 0.1689917405263497
MAPE: 82.22918498550546%
